## SPGD Algorithm. trying to reproduce DEEP CONTRAST Paper

In [3]:
!pip install xxhash
"""
"""
import sys
sys.path.insert(1, '../../hcipy')



In [ ]:
"""
"""
import sys
sys.path.insert(1, '../../hcipy')



## Necessary imports
import numpy as np
from hcipy import *
from hcipy.atmosphere import *
import matplotlib.pyplot as plt
from astropy.io import fits
import os, glob
import time

In [ ]:
## Create aperture and pupil/focal grids
wavelength = 532e-9
N = 512
D = 10.5e-3
pupil_grid = make_pupil_grid(N, D)
science_focal_grid = make_focal_grid(pupil_grid, 8, 20, wavelength)
wfs_focal_grid = make_focal_grid(pupil_grid, 8, 20, wavelength)
aperture = circular_aperture(D)

In [ ]:
# Telescope parameters
Dtel=4
tel_pupil_grid = make_pupil_grid(N, Dtel)
tel_aperture = circular_aperture(Dtel)

In [ ]:
## Create the deformable mirror
actuator_grid = make_pupil_grid(25, D*1.1)
sigma = D/25
gaussian_basis = make_gaussian_pokes(pupil_grid, actuator_grid, sigma)
dm = DeformableMirror(gaussian_basis)
num_modes = len(dm.influence_functions)
dm.actuators = np.zeros(num_modes)

In [ ]:
# Atmosphere parameters
pixels_per_frame = 1
velocity = np.array([pixels_per_frame,0])
L0 = 40
r0 = 0.2
height = 0

# Make atmosphere
np.random.seed(42)
#layers = [InfiniteAtmosphericLayer(tel_pupil_grid, Cn_squared_from_fried_parameter(r0, 500e-9), L0, velocity * tel_pupil_grid.delta[0], height, 2)]
layers = []
layer = InfiniteAtmosphericLayer(tel_pupil_grid, Cn_squared_from_fried_parameter(r0, 500e-9), L0, velocity * tel_pupil_grid.delta[0], height, 2)
layer2 = ModalAdaptiveOpticsLayer(layer, dm.influence_functions, 1)
layers.append(layer2)
atmosphere = MultiLayerAtmosphere(layers, False)

# Make initial phasescreen
wf_tel = Wavefront(tel_aperture(tel_pupil_grid), wavelength)
atms_time = 1
atmosphere.evolve_until(atms_time)
atms_time += 1
atmosphere.evolve_until(atms_time)
wfatms_tel = atmosphere.forward(wf_tel)

In [ ]:

## Create propagator from pupil to focal plane
prop = FraunhoferPropagator(pupil_grid, science_focal_grid, wavelength)

In [ ]:
## Create detector
flatfield = 0.05 # = 5% flat field error 
darkcurrentrate = 2 # = dark current counts per second
readnoise = 100 # = rms counts per read out
photonnoise = True

#Creating our detector.
#science_camera = NoisyDetector(input_grid=science_focal_grid, include_photon_noise=photonnoise, flat_field=flatfield, dark_current_rate=darkcurrentrate, read_noise=readnoise)
science_camera = NoiselessDetector()
shack_hartmann_camera = NoiselessDetector()

In [ ]:
## Create the Shack-Hartmann Wavefront sensor
F_mla = 40. / 0.3
N_mla = 22
D_mla = 10.5e-3
shwfs = SquareShackHartmannWavefrontSensorOptics(pupil_grid, F_mla, N_mla, D_mla)
shwfse = ShackHartmannWavefrontSensorEstimator(shwfs.mla_grid, shwfs.micro_lens_array.mla_index)

In [ ]:
## Demagnify wavefront and phase-screen for the optics
mag = Magnifier(10.5e-3/4)
wf = mag.forward(wf_tel)
wfatms = mag.forward(wfatms_tel)

## Generate a diffraction limited image for metrics
diff_lim_img = prop(wf).power

In [ ]:
## Get the unit lambda/D
l_D = wavelength / D
plot_grid = make_focal_grid(make_pupil_grid(512), 8, 20)

In [ ]:
## Create a noiseless camera image from the perfectly flat wavefront with coronograph
wfdm = dm.forward(wf)
imapp = prop(wfdm).power


In [ ]:
########### The test starts here #############################################

# Data to keep track of
cost_function = []
strehl_evol = []
contrast_evol = []
modes_evolution = []
lrate_evol = []

In [ ]:
# Reset the learning rate, gradients, and perturbatory sequence
lrate = 1e-6
amp = 1e-10
iterations = 10000
merit = 999.99e9
grads = np.zeros(dm.actuators.size,)
np.random.seed(42)

In [ ]:
# Start inner loop closed-loop control here
for loop in np.arange(iterations):
    dm_wf = dm.forward(wfatms)
    
    # Science optical path here
    ncp_wf = ncp.forward(dm_wf)
    science_camera.integrate(prop(ncp_wf), 1, 1)
    sci_img = science_camera.read_out()
    
    # Wavefront sensor optical path here
    # Implement spatial filter to simulate reconstructed wavefront
    reconstruct_wf = prop.backward(spatial_filter.forward(prop(dm_wf)))
    
    # Control path here
    wfout = app.forward(reconstruct_wf)
    test_img = prop(wfout).power
    
    test_strehl = test_img[np.argmax(diff_lim_img)] / diff_lim_img.max()
    test_contrast = test_img[dz_ind].mean() / diff_lim_img.max()
    new_merit = np.sqrt(test_contrast) / test_strehl
    
    # If the update in the cost function is small, or the merit has worsened
    if merit - new_merit < 1e-8:
        # If there has been an improvement (decrease) in the cost function
        # Then increase the learning rate
        if new_merit < merit:
            lrate *= 1.05
        # Otherwise decrease learning rate and go back to previous actuator values
        else:
            dm.actuators = modes_evolution[-1].copy()
            new_merit = merit
            lrate *= 0.7
    
    merit = new_merit
    cost_function.append(merit)
    
    strehl_evol.append(test_strehl)
    contrast_evol.append(test_contrast)
    modes_evolution.append(dm.actuators.copy())
    lrate_evol.append(lrate)
    
    if loop % 1000 == 0:
        print("Loop {0}: Calculated strehl: {1:.4f} contrast: {2:.4E} cost: {3:.4E}\n"\
              .format(loop, test_strehl, test_contrast, merit))
    
    # Make a measurement of the metric with DM actuators with delta added
    dm_sequence = np.random.choice([-1,1], dm.actuators.shape) * amp
    dm.actuators += dm_sequence
    dm_wf = dm.forward(wfatms)
    reconstruct_wf = prop.backward(spatial_filter.forward(prop(dm_wf)))
    wfout = app.forward(reconstruct_wf)
    test_img = prop(wfout).power
    
    test_strehl = test_img[np.argmax(diff_lim_img)] / diff_lim_img.max()
    test_contrast = test_img[dz_ind].mean() / diff_lim_img.max()
    merit_delta = np.sqrt(test_contrast) / test_strehl
    
    # Take the difference between measurements
    # This is equivalent to having a low pass filter of the cost function
    LF = merit_delta - merit
    
    # Multiply by the delta amplitudes vector and divide by amp^2 to get gradient
    grads = (LF * dm_sequence)# / (amp ** 2)
    
    # Calculate new DM actuator positions using learning rate and gradient
    dm.actuators -= lrate * grads

In [ ]:


#from utils.shack_hartmann_calibrator import shack_hartmann_calibrator












## Get the app coronagraph
app_amp = 1
app_phase = 0
app = Apodizer(app_amp * np.exp(1j * app_phase))



## Create a spatial filter
filt_aperture = circular_aperture(25)
spatial_filter = Apodizer(filt_aperture(science_focal_grid))








# Create an NCP aberration
num_coeffs = 40
plaw_index = -1.
np.random.seed(42)
coeffs = ((np.random.rand(num_coeffs) - 0.5) * 2 ) * (np.arange(num_coeffs) + 1) ** plaw_index
coeffs = np.zeros(coeffs.shape)
zernike_basis = make_zernike_basis(num_coeffs, D, pupil_grid, 2)
ncp_phase = np.dot(zernike_basis.transformation_matrix, coeffs)
ncp = Apodizer(np.exp(1j * ncp_phase))







# Save dm actuators after loop 1
dm_loop1 = dm.actuators.copy()
test_strehl_loop1 = test_strehl
test_contrast_loop1 = test_contrast
postloop_lrate = lrate

wf_loop1 = dm_wf.copy()
sci_img1 = sci_img.copy()

phasescreens = []
phasescreens.append(wfatms.copy())
# Now run the loop while tracking atmospheric changes
for opt_loop in np.arange(5e4):
    # Update the atmospheric turbulence
    if opt_loop % 100 == 0:
        print("Phasescreen changes\n")
        atms_time += 1
        atmosphere.evolve_until(atms_time)
        wfatms_tel = atmosphere.forward(wf_tel)
        wfatms = mag.forward(wfatms_tel)
        phasescreens.append(wfatms)
        lrate = 1e-6
        merit = 999.99e9
        #dm.actuators = np.zeros(dm.actuators.shape)
    
    # Common path here
    dm_wf = dm.forward(wfatms)
    
    # Science optical path here
    ncp_wf = ncp.forward(dm_wf)
    app_wf = app.forward(ncp_wf)
    science_camera.integrate(prop(app_wf), dt=1e-2)
    
    # Wavefront sensor optical path here
    # Implement spatial filter to simulate reconstructed wavefront
    reconstruct_wf = prop.backward(spatial_filter.forward(prop(dm_wf)))

    # Control path here
    wfout = app.forward(reconstruct_wf)
    test_img = prop(wfout).power
    
    test_strehl = test_img[np.argmax(diff_lim_img)] / diff_lim_img.max()
    test_contrast = test_img[dz_ind].mean() / diff_lim_img.max()
    new_merit = np.sqrt(test_contrast) / test_strehl
    
    # If the update in the cost function is small
    if merit - new_merit < 1e-8:
        # If there has been an improvement (decrease) in the cost function
        # Then increase the learning rate
        if new_merit < merit:
            lrate *= 1.05
        # Otherwise decrease learning rate and go back to previous actuator values
        else:
            dm.actuators = modes_evolution[-1].copy()
            new_merit = merit
            lrate *= 0.7
            
    merit = new_merit
    
    cost_function.append(merit)
    strehl_evol.append(test_strehl)
    contrast_evol.append(test_contrast)
    lrate_evol.append(lrate)
    
    modes_evolution.append(dm.actuators.copy())
    
    if opt_loop % 10 == 0:
        print("Tracking loop {0}: Calculated strehl: {1:.4f} contrast: {2:.4E} cost: {3:.4E}\n"\
              .format(opt_loop, test_strehl, test_contrast, merit))
                
    # Make a measurement of the metric with DM actuators with delta added
    dm_sequence = np.random.choice([-1,1], dm.actuators.shape) * amp
    dm.actuators += dm_sequence 
    dm_wf = dm.forward(wfatms)
    reconstruct_wf = prop.backward(spatial_filter.forward(prop(dm_wf)))
    wfout = app.forward(reconstruct_wf)
    test_img = prop(wfout).power
    
    test_strehl = test_img[np.argmax(diff_lim_img)] / diff_lim_img.max()
    test_contrast = test_img[dz_ind].mean() / diff_lim_img.max()
    merit_delta = np.sqrt(test_contrast) / test_strehl
    
    # Take the difference between measurements
    # This is equivalent to having a low pass filter of the cost function
    LF = merit_delta - merit
    
    # Multiply by the delta amplitudes vector and divide by amp^2 to get gradient
    grads = (LF * dm_sequence) #/ (amp ** 2)
    
    # Calculate new DM actuator positions using learning rate and gradient
    dm.actuators -= lrate * grads

final_sci_img = science_camera.read_out()

In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Nov  8 14:58:34 2018
Stochastic Parallel Gradient Descent optimizer code for optimizing an AO + coronagraph
optical imaging system. In this test, a nested optimizer is run within another optimizer
loop. The inner optimizer makes use of the Shack-Hartmann wavefront sensor data to optimize
contrast at the virtual, calculated conjugate image plane. This accounts for residual
turbulence and common path aberrations. The outer loop uses a similar optimizer, but optimizes
contrast at the actual science image plane. This accounts for non common path errors
and wavefront reconstruction errors.
@author: vikram
"""

## Necessary imports
import numpy as np
from hcipy import *
from hcipy.atmosphere import *
import matplotlib.pyplot as plt
from astropy.io import fits
import os, glob
import time

#from utils.shack_hartmann_calibrator import shack_hartmann_calibrator

## Create aperture and pupil/focal grids
wavelength = 532e-9
N = 512
D = 10.5e-3
pupil_grid = make_pupil_grid(N, D)
science_focal_grid = make_focal_grid(pupil_grid, 8, 20, wavelength)
wfs_focal_grid = make_focal_grid(pupil_grid, 8, 20, wavelength)
aperture = circular_aperture(D)

# Telescope parameters
Dtel=4
tel_pupil_grid = make_pupil_grid(N, Dtel)
tel_aperture = circular_aperture(Dtel)

## Create the deformable mirror
actuator_grid = make_pupil_grid(25, D*1.1)
sigma = D/25
gaussian_basis = make_gaussian_pokes(pupil_grid, actuator_grid, sigma)
dm = DeformableMirror(gaussian_basis)
num_modes = len(dm.influence_functions)
dm.actuators = np.zeros(num_modes)

# Atmosphere parameters
pixels_per_frame = 1
velocity = np.array([pixels_per_frame,0])
L0 = 40
r0 = 0.2
height = 0

# Make atmosphere
np.random.seed(42)
#layers = [InfiniteAtmosphericLayer(tel_pupil_grid, Cn_squared_from_fried_parameter(r0, 500e-9), L0, velocity * tel_pupil_grid.delta[0], height, 2)]
layers = []
layer = InfiniteAtmosphericLayer(tel_pupil_grid, Cn_squared_from_fried_parameter(r0, 500e-9), L0, velocity * tel_pupil_grid.delta[0], height, 2)
layer2 = ModalAdaptiveOpticsLayer(layer, dm.influence_functions, 1)
layers.append(layer2)
atmosphere = MultiLayerAtmosphere(layers, False)

# Make initial phasescreen
wf_tel = Wavefront(tel_aperture(tel_pupil_grid), wavelength)
atms_time = 1
atmosphere.evolve_until(atms_time)
atms_time += 1
atmosphere.evolve_until(atms_time)
wfatms_tel = atmosphere.forward(wf_tel)

## Demagnify wavefront and phase-screen for the optics
mag = Magnifier(10.5e-3/4)
wf = mag.forward(wf_tel)
wfatms = mag.forward(wfatms_tel)

## Create propagator from pupil to focal plane
prop = FraunhoferPropagator(pupil_grid, science_focal_grid, wavelength)

## Get the app coronagraph
app_amp = 1
app_phase = 0
app = Apodizer(app_amp * np.exp(1j * app_phase))

## Create detector
flatfield = 0.05 # = 5% flat field error 
darkcurrentrate = 2 # = dark current counts per second
readnoise = 100 # = rms counts per read out
photonnoise = True

#Creating our detector.
#science_camera = NoisyDetector(input_grid=science_focal_grid, include_photon_noise=photonnoise, flat_field=flatfield, dark_current_rate=darkcurrentrate, read_noise=readnoise)
science_camera = NoiselessDetector()
shack_hartmann_camera = NoiselessDetector()

## Create a spatial filter
filt_aperture = circular_aperture(25)
spatial_filter = Apodizer(filt_aperture(science_focal_grid))

## Create the Shack-Hartmann Wavefront sensor
F_mla = 40. / 0.3
N_mla = 22
D_mla = 10.5e-3
shwfs = SquareShackHartmannWavefrontSensorOptics(pupil_grid, F_mla, N_mla, D_mla)
shwfse = ShackHartmannWavefrontSensorEstimator(shwfs.mla_grid, shwfs.micro_lens_array.mla_index)

## Generate a diffraction limited image for metrics
diff_lim_img = prop(wf).power

## Get the unit lambda/D
l_D = wavelength / D
plot_grid = make_focal_grid(make_pupil_grid(512), 8, 20)

## Create a noiseless camera image from the perfectly flat wavefront with coronograph
wfdm = dm.forward(wf)
wfapp = app.forward(wfdm)
imapp = prop(wfapp).power
dz_ind = np.where((imapp.grid.x >= (2 * l_D)) &\
                  (imapp.grid.x <= (8 * l_D)) &\
                  (imapp.grid.y >= (-3 * l_D)) &\
                  (imapp.grid.y <= (3 * l_D)))

# Create an NCP aberration
num_coeffs = 40
plaw_index = -1.
np.random.seed(42)
coeffs = ((np.random.rand(num_coeffs) - 0.5) * 2 ) * (np.arange(num_coeffs) + 1) ** plaw_index
coeffs = np.zeros(coeffs.shape)
zernike_basis = make_zernike_basis(num_coeffs, D, pupil_grid, 2)
ncp_phase = np.dot(zernike_basis.transformation_matrix, coeffs)
ncp = Apodizer(np.exp(1j * ncp_phase))

########### The test starts here #############################################

# Data to keep track of
cost_function = []
strehl_evol = []
contrast_evol = []
modes_evolution = []
lrate_evol = []

# Reset the learning rate, gradients, and perturbatory sequence
lrate = 1e-6
amp = 1e-10
iterations = 10000
merit = 999.99e9
grads = np.zeros(dm.actuators.size,)
np.random.seed(42)

# Start inner loop closed-loop control here
for loop in np.arange(iterations):
    dm_wf = dm.forward(wfatms)
    
    # Science optical path here
    ncp_wf = ncp.forward(dm_wf)
    app_wf = app.forward(ncp_wf)
    science_camera.integrate(prop(app_wf), 1, 1)
    sci_img = science_camera.read_out()
    
    # Wavefront sensor optical path here
    # Implement spatial filter to simulate reconstructed wavefront
    reconstruct_wf = prop.backward(spatial_filter.forward(prop(dm_wf)))
    
    # Control path here
    wfout = app.forward(reconstruct_wf)
    test_img = prop(wfout).power
    
    test_strehl = test_img[np.argmax(diff_lim_img)] / diff_lim_img.max()
    test_contrast = test_img[dz_ind].mean() / diff_lim_img.max()
    new_merit = np.sqrt(test_contrast) / test_strehl
    
    # If the update in the cost function is small, or the merit has worsened
    if merit - new_merit < 1e-8:
        # If there has been an improvement (decrease) in the cost function
        # Then increase the learning rate
        if new_merit < merit:
            lrate *= 1.05
        # Otherwise decrease learning rate and go back to previous actuator values
        else:
            dm.actuators = modes_evolution[-1].copy()
            new_merit = merit
            lrate *= 0.7
    
    merit = new_merit
    cost_function.append(merit)
    
    strehl_evol.append(test_strehl)
    contrast_evol.append(test_contrast)
    modes_evolution.append(dm.actuators.copy())
    lrate_evol.append(lrate)
    
    if loop % 1000 == 0:
        print("Loop {0}: Calculated strehl: {1:.4f} contrast: {2:.4E} cost: {3:.4E}\n"\
              .format(loop, test_strehl, test_contrast, merit))
    
    # Make a measurement of the metric with DM actuators with delta added
    dm_sequence = np.random.choice([-1,1], dm.actuators.shape) * amp
    dm.actuators += dm_sequence
    dm_wf = dm.forward(wfatms)
    reconstruct_wf = prop.backward(spatial_filter.forward(prop(dm_wf)))
    wfout = app.forward(reconstruct_wf)
    test_img = prop(wfout).power
    
    test_strehl = test_img[np.argmax(diff_lim_img)] / diff_lim_img.max()
    test_contrast = test_img[dz_ind].mean() / diff_lim_img.max()
    merit_delta = np.sqrt(test_contrast) / test_strehl
    
    # Take the difference between measurements
    # This is equivalent to having a low pass filter of the cost function
    LF = merit_delta - merit
    
    # Multiply by the delta amplitudes vector and divide by amp^2 to get gradient
    grads = (LF * dm_sequence)# / (amp ** 2)
    
    # Calculate new DM actuator positions using learning rate and gradient
    dm.actuators -= lrate * grads

# Save dm actuators after loop 1
dm_loop1 = dm.actuators.copy()
test_strehl_loop1 = test_strehl
test_contrast_loop1 = test_contrast
postloop_lrate = lrate

wf_loop1 = dm_wf.copy()
sci_img1 = sci_img.copy()

phasescreens = []
phasescreens.append(wfatms.copy())
# Now run the loop while tracking atmospheric changes
for opt_loop in np.arange(5e4):
    # Update the atmospheric turbulence
    if opt_loop % 100 == 0:
        print("Phasescreen changes\n")
        atms_time += 1
        atmosphere.evolve_until(atms_time)
        wfatms_tel = atmosphere.forward(wf_tel)
        wfatms = mag.forward(wfatms_tel)
        phasescreens.append(wfatms)
        lrate = 1e-6
        merit = 999.99e9
        #dm.actuators = np.zeros(dm.actuators.shape)
    
    # Common path here
    dm_wf = dm.forward(wfatms)
    
    # Science optical path here
    ncp_wf = ncp.forward(dm_wf)
    app_wf = app.forward(ncp_wf)
    science_camera.integrate(prop(app_wf), dt=1e-2)
    
    # Wavefront sensor optical path here
    # Implement spatial filter to simulate reconstructed wavefront
    reconstruct_wf = prop.backward(spatial_filter.forward(prop(dm_wf)))

    # Control path here
    wfout = app.forward(reconstruct_wf)
    test_img = prop(wfout).power
    
    test_strehl = test_img[np.argmax(diff_lim_img)] / diff_lim_img.max()
    test_contrast = test_img[dz_ind].mean() / diff_lim_img.max()
    new_merit = np.sqrt(test_contrast) / test_strehl
    
    # If the update in the cost function is small
    if merit - new_merit < 1e-8:
        # If there has been an improvement (decrease) in the cost function
        # Then increase the learning rate
        if new_merit < merit:
            lrate *= 1.05
        # Otherwise decrease learning rate and go back to previous actuator values
        else:
            dm.actuators = modes_evolution[-1].copy()
            new_merit = merit
            lrate *= 0.7
            
    merit = new_merit
    
    cost_function.append(merit)
    strehl_evol.append(test_strehl)
    contrast_evol.append(test_contrast)
    lrate_evol.append(lrate)
    
    modes_evolution.append(dm.actuators.copy())
    
    if opt_loop % 10 == 0:
        print("Tracking loop {0}: Calculated strehl: {1:.4f} contrast: {2:.4E} cost: {3:.4E}\n"\
              .format(opt_loop, test_strehl, test_contrast, merit))
                
    # Make a measurement of the metric with DM actuators with delta added
    dm_sequence = np.random.choice([-1,1], dm.actuators.shape) * amp
    dm.actuators += dm_sequence 
    dm_wf = dm.forward(wfatms)
    reconstruct_wf = prop.backward(spatial_filter.forward(prop(dm_wf)))
    wfout = app.forward(reconstruct_wf)
    test_img = prop(wfout).power
    
    test_strehl = test_img[np.argmax(diff_lim_img)] / diff_lim_img.max()
    test_contrast = test_img[dz_ind].mean() / diff_lim_img.max()
    merit_delta = np.sqrt(test_contrast) / test_strehl
    
    # Take the difference between measurements
    # This is equivalent to having a low pass filter of the cost function
    LF = merit_delta - merit
    
    # Multiply by the delta amplitudes vector and divide by amp^2 to get gradient
    grads = (LF * dm_sequence) #/ (amp ** 2)
    
    # Calculate new DM actuator positions using learning rate and gradient
    dm.actuators -= lrate * grads

final_sci_img = science_camera.read_out()

/home/rahulbhadani/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: This functions signature is deprecated and will be removed with the next major update. Use the updated function signature or call make_focal_grid_from_pupil_grid() directly instead.
/home/rahulbhadani/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning: This functions signature is deprecated and will be removed with the next major update. Use the updated function signature or call make_focal_grid_from_pupil_grid() directly instead.
/home/rahulbhadani/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:112: DeprecationWarning: This functions signature is deprecated and will be removed with the next major update. Use the updated function signature or call make_focal_grid_from_pupil_grid() directly instead.


Loop 0: Calculated strehl: 7897106371569672781824.0000 contrast: 1.1103E+16 cost: 1.3343E-14

Loop 1000: Calculated strehl: 7764535176707389259776.0000 contrast: 4.7854E+15 cost: 8.9052E-15

Loop 2000: Calculated strehl: 7542291527369773547520.0000 contrast: 3.7483E+15 cost: 8.1107E-15

Loop 3000: Calculated strehl: 7363512211508223279104.0000 contrast: 3.1255E+15 cost: 7.5899E-15

Loop 4000: Calculated strehl: 7127180224669327294464.0000 contrast: 2.6292E+15 cost: 7.1910E-15

Loop 5000: Calculated strehl: 7033374206240473743360.0000 contrast: 2.3789E+15 cost: 6.9347E-15

Loop 6000: Calculated strehl: 6936776540980625014784.0000 contrast: 2.1839E+15 cost: 6.7362E-15

Loop 7000: Calculated strehl: 6837492223022435663872.0000 contrast: 2.0139E+15 cost: 6.5618E-15

Loop 8000: Calculated strehl: 6777132159822225997824.0000 contrast: 1.9115E+15 cost: 6.4512E-15

Loop 9000: Calculated strehl: 6716476717086864834560.0000 contrast: 1.8156E+15 cost: 6.3432E-15

Phasescreen changes

Tracking loo

Tracking loop 670.0: Calculated strehl: 6639774250527534088192.0000 contrast: 2.2011E+15 cost: 7.0600E-15

Tracking loop 680.0: Calculated strehl: 6644288060261562056704.0000 contrast: 2.1998E+15 cost: 7.0577E-15

Tracking loop 690.0: Calculated strehl: 6649102676793152765952.0000 contrast: 2.1999E+15 cost: 7.0497E-15

Phasescreen changes

Tracking loop 700.0: Calculated strehl: 6640960370793930293248.0000 contrast: 2.2608E+15 cost: 7.1598E-15

Tracking loop 710.0: Calculated strehl: 6648705686696956526592.0000 contrast: 2.2641E+15 cost: 7.1524E-15

Tracking loop 720.0: Calculated strehl: 6646739517759371083776.0000 contrast: 2.2572E+15 cost: 7.1478E-15

Tracking loop 730.0: Calculated strehl: 6659416860381476290560.0000 contrast: 2.2643E+15 cost: 7.1416E-15

Tracking loop 740.0: Calculated strehl: 6651013389356353191936.0000 contrast: 2.2535E+15 cost: 7.1373E-15

Tracking loop 750.0: Calculated strehl: 6651349825668294639616.0000 contrast: 2.2513E+15 cost: 7.1323E-15

Tracking loop 76

Tracking loop 1420.0: Calculated strehl: 6642298624346004914176.0000 contrast: 2.7288E+15 cost: 7.8644E-15

Tracking loop 1430.0: Calculated strehl: 6641094347321222627328.0000 contrast: 2.7244E+15 cost: 7.8595E-15

Tracking loop 1440.0: Calculated strehl: 6644125533179244183552.0000 contrast: 2.7237E+15 cost: 7.8481E-15

Tracking loop 1450.0: Calculated strehl: 6648534896151605805056.0000 contrast: 2.7196E+15 cost: 7.8418E-15

Tracking loop 1460.0: Calculated strehl: 6643327677691936636928.0000 contrast: 2.7073E+15 cost: 7.8322E-15

Tracking loop 1470.0: Calculated strehl: 6647561629317082382336.0000 contrast: 2.7041E+15 cost: 7.8226E-15

Tracking loop 1480.0: Calculated strehl: 6643905427975332429824.0000 contrast: 2.6928E+15 cost: 7.8105E-15

Tracking loop 1490.0: Calculated strehl: 6639497194556225486848.0000 contrast: 2.6845E+15 cost: 7.8036E-15

Phasescreen changes

Tracking loop 1500.0: Calculated strehl: 6641567073288978432000.0000 contrast: 2.8137E+15 cost: 7.9867E-15

Trackin

Tracking loop 2170.0: Calculated strehl: 6636882317361944723456.0000 contrast: 3.0575E+15 cost: 8.3314E-15

Tracking loop 2180.0: Calculated strehl: 6635840282374399066112.0000 contrast: 3.0558E+15 cost: 8.3264E-15

Tracking loop 2190.0: Calculated strehl: 6636002621218752561152.0000 contrast: 3.0404E+15 cost: 8.3092E-15

Phasescreen changes

Tracking loop 2200.0: Calculated strehl: 6640491045754740146176.0000 contrast: 3.1329E+15 cost: 8.4290E-15

Tracking loop 2210.0: Calculated strehl: 6639565619432906555392.0000 contrast: 3.1302E+15 cost: 8.4232E-15

Tracking loop 2220.0: Calculated strehl: 6640517557809618878464.0000 contrast: 3.1192E+15 cost: 8.4105E-15

Tracking loop 2230.0: Calculated strehl: 6635499092322467971072.0000 contrast: 3.1118E+15 cost: 8.4050E-15

Tracking loop 2240.0: Calculated strehl: 6637290213957148082176.0000 contrast: 3.1088E+15 cost: 8.4006E-15

Tracking loop 2250.0: Calculated strehl: 6637773522780054093824.0000 contrast: 3.1037E+15 cost: 8.3929E-15

Trackin

...

Tracking loop 46670.0: Calculated strehl: 7176756504565924233216.0000 contrast: 2.9362E+15 cost: 7.5479E-15

Tracking loop 46680.0: Calculated strehl: 7175211107656576008192.0000 contrast: 2.9268E+15 cost: 7.5398E-15

Tracking loop 46690.0: Calculated strehl: 7181649339440041033728.0000 contrast: 2.9269E+15 cost: 7.5332E-15

Phasescreen changes

Tracking loop 46700.0: Calculated strehl: 7178658571265947729920.0000 contrast: 3.0219E+15 cost: 7.6576E-15

Tracking loop 46710.0: Calculated strehl: 7176927250691713400832.0000 contrast: 3.0189E+15 cost: 7.6502E-15

Tracking loop 46720.0: Calculated strehl: 7177757869697353646080.0000 contrast: 3.0073E+15 cost: 7.6388E-15

Tracking loop 46730.0: Calculated strehl: 7178522029385526018048.0000 contrast: 2.9970E+15 cost: 7.6262E-15

Tracking loop 46740.0: Calculated strehl: 7176562089434165542912.0000 contrast: 2.9895E+15 cost: 7.6188E-15

Tracking loop 46750.0: Calculated strehl: 7176581938553729056768.0000 contrast: 2.9856E+15 cost: 7.6137E-15

Tracking loop 47410.0: Calculated strehl: 7246242628697219661824.0000 contrast: 2.9132E+15 cost: 7.4486E-15

Tracking loop 47420.0: Calculated strehl: 7251189378683546632192.0000 contrast: 2.9127E+15 cost: 7.4428E-15

Tracking loop 47430.0: Calculated strehl: 7243817583554691334144.0000 contrast: 2.9027E+15 cost: 7.4371E-15

Tracking loop 47440.0: Calculated strehl: 7240295822741467037696.0000 contrast: 2.9007E+15 cost: 7.4313E-15

Tracking loop 47450.0: Calculated strehl: 7236730612823152918528.0000 contrast: 2.8851E+15 cost: 7.4223E-15

Tracking loop 47460.0: Calculated strehl: 7237053156640219463680.0000 contrast: 2.8856E+15 cost: 7.4210E-15

Tracking loop 47470.0: Calculated strehl: 7232116246286070448128.0000 contrast: 2.8738E+15 cost: 7.4088E-15

Tracking loop 47480.0: Calculated strehl: 7230623045424683941888.0000 contrast: 2.8666E+15 cost: 7.4047E-15

Tracking loop 47490.0: Calculated strehl: 7229766665327558524928.0000 contrast: 2.8640E+15 cost: 7.4022E-15

Phasescreen changes

Tracking loop 48150.0: Calculated strehl: 7167403790108166455296.0000 contrast: 3.2295E+15 cost: 7.9255E-15

Tracking loop 48160.0: Calculated strehl: 7167590982977891336192.0000 contrast: 3.2172E+15 cost: 7.9114E-15

Tracking loop 48170.0: Calculated strehl: 7168586740498670026752.0000 contrast: 3.2151E+15 cost: 7.9047E-15

Tracking loop 48180.0: Calculated strehl: 7171048825757528752128.0000 contrast: 3.2042E+15 cost: 7.8910E-15

Tracking loop 48190.0: Calculated strehl: 7171409192564242251776.0000 contrast: 3.1957E+15 cost: 7.8778E-15

Phasescreen changes

Tracking loop 48200.0: Calculated strehl: 7162898807402951344128.0000 contrast: 3.2310E+15 cost: 7.9356E-15

Tracking loop 48210.0: Calculated strehl: 7158978882493940236288.0000 contrast: 3.2248E+15 cost: 7.9323E-15

Tracking loop 48220.0: Calculated strehl: 7168127353015794401280.0000 contrast: 3.2230E+15 cost: 7.9199E-15

Tracking loop 48230.0: Calculated strehl: 7165339045448788213760.0000 contrast: 3.2138E+15 cost: 7.9096E-15

Tracking loop 48890.0: Calculated strehl: 7126156938351330983936.0000 contrast: 3.1927E+15 cost: 7.9291E-15

Phasescreen changes

Tracking loop 48900.0: Calculated strehl: 7118823183213338296320.0000 contrast: 3.2609E+15 cost: 8.0215E-15

Tracking loop 48910.0: Calculated strehl: 7113510589667777445888.0000 contrast: 3.2532E+15 cost: 8.0158E-15

Tracking loop 48920.0: Calculated strehl: 7120750626214720307200.0000 contrast: 3.2467E+15 cost: 8.0020E-15

Tracking loop 48930.0: Calculated strehl: 7121150825928040906752.0000 contrast: 3.2494E+15 cost: 7.9958E-15

Tracking loop 48940.0: Calculated strehl: 7115364487279832727552.0000 contrast: 3.2270E+15 cost: 7.9837E-15

Tracking loop 48950.0: Calculated strehl: 7122730560826176962560.0000 contrast: 3.2296E+15 cost: 7.9787E-15

Tracking loop 48960.0: Calculated strehl: 7123692464688127279104.0000 contrast: 3.2283E+15 cost: 7.9710E-15

Tracking loop 48970.0: Calculated strehl: 7122295619997892870144.0000 contrast: 3.2176E+15 cost: 7.9631E-15

Tracking loop 49630.0: Calculated strehl: 7137728338843523874816.0000 contrast: 3.0420E+15 cost: 7.7263E-15

Tracking loop 49640.0: Calculated strehl: 7139889151188982038528.0000 contrast: 3.0364E+15 cost: 7.7177E-15

Tracking loop 49650.0: Calculated strehl: 7141020075829260976128.0000 contrast: 3.0284E+15 cost: 7.7063E-15

Tracking loop 49660.0: Calculated strehl: 7138276657768536473600.0000 contrast: 3.0212E+15 cost: 7.6983E-15

Tracking loop 49670.0: Calculated strehl: 7145568782371863396352.0000 contrast: 3.0201E+15 cost: 7.6878E-15

Tracking loop 49680.0: Calculated strehl: 7138248383631420555264.0000 contrast: 3.0088E+15 cost: 7.6826E-15

Tracking loop 49690.0: Calculated strehl: 7146163237369759662080.0000 contrast: 3.0133E+15 cost: 7.6776E-15

Phasescreen changes

Tracking loop 49700.0: Calculated strehl: 7142270037575392559104.0000 contrast: 3.0471E+15 cost: 7.7287E-15

Tracking loop 49710.0: Calculated strehl: 7144227869393825038336.0000 contrast: 3.0375E+15 cost: 7.7144E-15